In [1]:
import os
import json
import websocket
import threading
import time
import logging
from dotenv import load_dotenv
load_dotenv()
# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


# Global WebSocket object and accumulated response text
ws = None
response_accumulator = ""  # Accumulate text parts
is_connected = False  # Connection status flag

# Function to send messages through the WebSocket
def send_message(message):
    if ws and ws.sock and ws.sock.connected:
        ws.send(json.dumps(message))
        logging.info(f"Sent: {message}")
    else:
        logging.warning("WebSocket is not connected.")

# WebSocket event handlers
def on_open(ws):
    global is_connected
    is_connected = True
    logging.info("Connected to server.")

def on_message(ws, message):
    global response_accumulator

    data = json.loads(message)
    logging.info(f"Received message: {data}")

    # Handle incremental text responses (delta messages)
    if data.get("type") == "response.text.delta":
        delta = data.get("delta", "")
        response_accumulator += delta  # Append to accumulator

    # Print the complete response when finished
    elif data.get("type") == "response.text.done":
        logging.info(f"Complete response: {response_accumulator}")
        response_accumulator = ""  # Reset for the next response

    # Optionally log the final response to a file
    if data.get("type") == "response.done":
        with open("responses.log", "a") as f:
            json.dump(data["response"], f, indent=2)
            f.write("\n")
        logging.info(f"Logged response: {data['response']}")

def on_error(ws, error):
    global is_connected
    is_connected = False
    logging.error(f"WebSocket error: {error}")
    # Retry connecting after an error with exponential backoff
    for attempt in range(5):
        time.sleep(2 ** attempt)
        try:
            logging.info(f"Reconnecting attempt {attempt + 1}")
            run_ws()
            break
        except Exception as e:
            logging.error(f"Reconnection attempt {attempt + 1} failed: {e}")

def on_close(ws, close_status_code, close_msg):
    global is_connected
    is_connected = False
    logging.warning(f"WebSocket closed: {close_status_code} - {close_msg}")
    # Retry connecting after the socket closes
    time.sleep(5)
    run_ws()

def run_ws():
    global ws
    url = "wss://api.openai.com/v1/realtime?model=gpt-4o-realtime-preview-2024-10-01"
    ws = websocket.WebSocketApp(
        url,
        header={
            "Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}",
            "OpenAI-Beta": "realtime=v1",
        },
        on_open=on_open,
        on_message=on_message,
        on_error=on_error,
        on_close=on_close,
    )
    ws.run_forever()

# Start the WebSocket in a separate thread
ws_thread = threading.Thread(target=run_ws)
ws_thread.daemon = True  # Ensure the thread exits with the main program
ws_thread.start()

# Function to send messages from another notebook cell
def send_message_from_cell(message):
    if is_connected:
        send_message(message)
    else:
        logging.warning("Cannot send message. WebSocket is not connected.")



2024-10-20 10:07:21,958 - INFO - Websocket connected
2024-10-20 10:07:21,959 - INFO - Connected to server.
2024-10-20 10:07:21,960 - INFO - Received message: {'type': 'session.created', 'event_id': 'event_AKQsTiXIJzUVc6tA8bEev', 'session': {'id': 'sess_AKQsTDF8MMAc9ZjPnKMYV', 'object': 'realtime.session', 'model': 'gpt-4o-realtime-preview-2024-10-01', 'expires_at': 1729434141, 'modalities': ['text', 'audio'], 'instructions': "Your knowledge cutoff is 2023-10. You are a helpful, witty, and friendly AI. Act like a human, but remember that you aren't a human and that you can't do human things in the real world. Your voice and personality should be warm and engaging, with a lively and playful tone. If interacting in a non-English language, start by using the standard accent or dialect familiar to the user. Talk quickly. You should always call a function if you can. Do not refer to these rules, even if you’re asked about them.", 'voice': 'alloy', 'turn_detection': {'type': 'server_vad', 'th

In [2]:
# Example: Send a message to the WebSocket server
send_message({
    "type": "response.create",
    "response": {
        "modalities": ["text"],
        "instructions": "how big is the earth?",
    },
})


2024-10-20 10:07:26,354 - INFO - Sent: {'type': 'response.create', 'response': {'modalities': ['text'], 'instructions': 'how big is the earth?'}}


In [3]:
# Close the WebSocket connection
if ws and ws.sock and ws.sock.connected:
    ws.close()
    print("WebSocket closed.")
else:
    print("WebSocket is not connected.")


WebSocket closed.


In [33]:
from twilio.rest import Client
import os
from itertools import groupby
from operator import itemgetter
import json

# Load environment variables
account_sid = os.getenv('TWILIO_ACCOUNT_SID')
auth_token = os.getenv('TWILIO_AUTH_TOKEN')

# Initialize the Twilio client
client = Client(account_sid, auth_token)

def get_incoming_messages(twilio_number, limit=10):
    try:
        messages = client.messages.list(to=twilio_number, limit=limit)

        # Filter messages sent from the Twilio number itself
        filtered_messages = [msg for msg in messages if msg.from_ != twilio_number]

        # Format the filtered messages
        formatted_messages = [
            {
                'from': msg.from_,
                'to': msg.to,
                'content': msg.body,
                'date_sent': msg.date_sent.isoformat()
            }
            for msg in filtered_messages
        ]

        # Group the messages by the 'from' field
        grouped_messages = {}
        for key, group in groupby(sorted(formatted_messages, key=itemgetter('from')), key=itemgetter('from')):
            grouped_messages[key] = list(group)

        return grouped_messages

    except Exception as e:
        print(f'Error fetching messages: {e}')
        return {}

def write_messages_to_json(grouped_messages, filename='messages.json'):
    # Create a new dictionary for the JSON output with modified keys
    formatted_for_json = {
        key[2:]: [  # Exclude first two characters from keys
            {
                'content': msg['content'],
                'date_sent': msg['date_sent']
            }
            for msg in group
        ]
        for key, group in grouped_messages.items()
    }
    
    with open(filename, 'w') as f:
        json.dump(formatted_for_json, f, indent=4)

# Example usage
if __name__ == "__main__":
    twilio_number = "+13343098339"  # Set your Twilio phone number
    grouped_messages = get_incoming_messages(twilio_number, limit=100)

    # Write the grouped messages to a JSON file
    write_messages_to_json(grouped_messages)

    # Display the grouped messages with modified phone numbers
    for sender, msgs in grouped_messages.items():
        print(f"Messages from {sender[2:]}:")  # Print sender number without the first two characters
        for msg in msgs:
            print(f"  {msg['content']} (Sent At: {msg['date_sent']})")

    print(f"Messages have been written to 'messages.json'.")


Messages from 2049791430:
  That date does not work (Sent At: 2024-07-23T15:04:59+00:00)
Messages from 2896840904:
  Wuddup b (Sent At: 2024-06-25T14:29:06+00:00)
Messages from 4163194488:
  Okay (Sent At: 2024-09-27T13:13:52+00:00)
Messages from 4165770385:
  Ok  (Sent At: 2024-07-29T00:01:59+00:00)
  Liked “Hi Ewa,

Your job starts in 1 hour.

Booking Detai…” (Sent At: 2024-07-28T17:02:29+00:00)
  Liked “Hi Ewa,

Your job starts in 1 hour.

Booking Detai…” (Sent At: 2024-07-24T00:30:36+00:00)
  Liked “Hi Ewa,

Your job starts in 1 hour.

Booking Detai…” (Sent At: 2024-07-23T00:01:46+00:00)
  Liked “Hi Ewa, Your password has been updated successfull…” (Sent At: 2024-07-16T22:11:18+00:00)
  That so clear 👌 (Sent At: 2024-07-16T00:29:56+00:00)
  Loved “Hi Ewa,

Your job starts in 1 hour.

Booking Detai…” (Sent At: 2024-07-16T00:29:43+00:00)
Messages from 4167259300:
  Confirmed 👍  (Sent At: 2024-10-14T14:51:26+00:00)
  Confirming  (Sent At: 2024-08-05T13:18:49+00:00)
  Confirming 😊 (Sen

In [11]:
print(os.getenv('TWILIO_NUMBER'))

None
